In [1]:
import plotly.express as px
import plotly.graph_objs as go
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash
import pandas as pd
import import_ipynb

from lat_lon_parser import parse

from ipynb.fs.full.MainApp import app


In [2]:
def extract_lon_lat(ResultListdataframe):
    
        ResultListdataframe['coord'] = ResultListdataframe['coord'].str.split("\(", expand=True)[1]
        ResultListdataframe['coord'] = ResultListdataframe['coord'].str.split("\)", expand=True)
        ResultListdataframe['lon'] = ResultListdataframe['coord'].str.split(" ", expand=True)[0].astype(float)
        ResultListdataframe['lat'] = ResultListdataframe['coord'].str.split(" ", expand=True)[1].astype(float)
        
        return ResultListdataframe
        
    

In [3]:
def plot_map_function(ResultListdataframe):
    
        fig = go.Figure()
        
        
        fig.add_trace(go.Scattergeo(
                        lat = ResultListdataframe['lat'],
                        lon = ResultListdataframe['lon'],
                        text = ResultListdataframe['lon'].astype(str),
                        mode = 'markers',
                                                         
        )
        )        
            
            
                    
        buttonlist = []
        for col in ResultListdataframe.columns:
                buttonlist.append(
                    dict(
                        args=['text', ResultListdataframe[col].astype(str)],
                        label=str(col),
                        method='restyle', 
                        )
                )
                    
#        scope_options = ["world", "europe"]
#        buttonlist_scope= []
#        for option in scope_options:
#            buttonlist_scope.append(
#                dict(
#                    args = ['scope', str(option)],
#                    label = str(option),
#                    method = 'relayout',
#            )
#        )
            
                    
        fig.update_geos(
            visible=True,
            resolution=110,
            showcountries=True, countrycolor="Black",
            showsubunits=True, subunitcolor="Blue",
            scope="world",
#            updatemenus=[
#                    dict(
#                        buttons=buttonlist_scope,
#                        direction="down",
#                        pad={"r": 10, "t": 10},
#                        showactive=True,
#                        x=-0.9,
#                        xanchor="left",
#                        y=0.60,
#                        yanchor="top"
#                        ), 
#            ]
          )

        fig.update_layout(
                autosize=True,
                updatemenus=[
                    dict(
                        buttons=buttonlist,
                        direction="down",
                        pad={"r": 10, "t": 10},
                        showactive=True,
                        x=-0.9,
                        xanchor="left",
                        y=0.90,
                        yanchor="top"
                        ), 

                     ],
            
                
                        
                    height = 500, 
                    #width = 1000,
                    margin={"r":300,"t":0,"l":0,"b":0}

                )
            
        return fig

In [4]:
#---------------------------------------------------Charts Function--------------------------------------------------------------
def gen_map_function(n_clicks, jsonified_ResultListdataframe):
    
    if n_clicks > 0:
        
        if jsonified_ResultListdataframe is None:
            
              return True,{ },{'display': 'none'}
        
        else:
       
            ResultListdataframe = pd.read_json(jsonified_ResultListdataframe, orient='split')
         
            if 'coord' in ResultListdataframe.columns:
            
                ResultListdataframe = extract_lon_lat(ResultListdataframe)
                fig = plot_map_function(ResultListdataframe)
        
            elif 'lon' in ResultListdataframe.columns: 
            
                fig = plot_map_function(ResultListdataframe)
            
            else:       
                return True,{ },{'display': 'none'}

            return False,fig,{'display': 'block'}
        
    return False,{ },{'display': 'none'}

In [5]:
#---------------------------------------------------Charts Callback--------------------------------------------------------------
@app.callback(
    [Output("alert-map", "is_open"),Output("map", "figure"), Output('result-map', 'style')],
    [Input("submit-btn", "n_clicks"),
    Input("intermediate-value", "children")],
)
def gen_map(n_clicks, jsonified_ResultListdataframe):
    return gen_map_function(n_clicks, jsonified_ResultListdataframe)
    
#---------------------------------------------------Charts Compare Callback--------------------------------------------------------------
@app.callback(
    [Output("alert-map2", "is_open"),Output("map2", "figure"), Output('result-map2', 'style')],
    [Input("compare-btn", "n_clicks"),
    Input("intermediate-value2", "children")] ,
)
def gen_map(n_clicks, jsonified_ResultListdataframe):
    return gen_map_function(n_clicks, jsonified_ResultListdataframe)